In [ ]:
from ga_ls import *

road_points = {
    0: (-1.064124, 110.685258),   
    1: (-1.919500, 110.671412),  
    2: (-0.625036, 110.427354)
}

routing = ClusterBasedDroneRouting_Hybrid(
    csv_file="ketapang-12-08-24_merged_clustered_3.csv",
    road_points=road_points,
    n_drones={0: 2, 1: 2, 2: 2}
)

routes = routing.optimize_all_clusters(routing)
routing.print_cluster_routes(routes)
routing.visualize_cluster_routes(routes)

Hyperparameter Tuning

In [ ]:
import optuna
from ga_ls import *

road_points = {
    0: (-1.064124, 110.685258),   
    1: (-1.919500, 110.671412),  
    2: (-0.625036, 110.427354)
}

def objective(trial):
    #Define hyperparameters for Bayesian optimization
    population_size = trial.suggest_int('population_size', 30, 100, step=10)
    generations = trial.suggest_int('generations', 100, 300, step=50)
    mutation_rate = trial.suggest_float('mutation_rate', 0.05, 0.3)
    local_search_freq = trial.suggest_int('local_search_freq', 5, 25, step=5)
    local_search_intensity = trial.suggest_int('local_search_intensity', 20, 100, step=10)

    #Initialize the routing class
    routing = ClusterBasedDroneRouting_Hybrid(
        csv_file="ketapang-12-08-24_merged_clustered_3.csv",
        road_points=road_points,
        n_drones={0: 2, 1: 2, 2: 2}
    )

    #Optimize using the trial hyperparameters
    routes = routing.optimize_all_clusters(
        population_size=population_size,
        generations=generations,
        mutation_rate=mutation_rate,
        local_search_freq=local_search_freq,
        local_search_intensity=local_search_intensity
    )

    #Compute total distance
    total_distance = 0
    for cluster_id, route_list in routes.items():
        for route in route_list:
            if len(route) < 2:
                continue
            for i in range(len(route) - 1):
                total_distance += routing.dist_matrix[route[i]][route[i + 1]]

    return total_distance 

# Run the optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=2000, show_progress_bar=True, n_jobs=-1)

# Print best result
print("Best hyperparameters:", study.best_params)
print("Best total distance:", study.best_value)